In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # 'last' 'none' ...

In [2]:
import pymrio
from pymrio.core.mriosystem import IOSystem as IOS
from pymrio.core.mriosystem import concate_extension
from pymrio.tools.iomath import div0
from pymrio.tools.iofunctions import *
from pymrio.tools.ioparser import *
import pandas as pd
import numpy as np
# import scipy.io
import scipy.sparse as sp 
from scipy.sparse import linalg as spla
import operator
import warnings
import matplotlib.pyplot as plt
from pylab import *
import pickle
from openpyxl import load_workbook
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # 'last' 'none' ...
warnings.filterwarnings('ignore')

In [3]:
path_io = '/mnt/dd/adrien/DD/Économie/Données/' # where the Exiobase data is
path_dl = '/home/adrien/Downloads/' # where the conso by age is, and where the impacts by age are stored

### Load data from already executed notebook

In [4]:
exio3 = pickle.load(open(path_io+'Exiobase 3.4/exio3.pkl', 'rb'))

In [ ]:
impacts_FR = pickle.load(open(path_io + 'Exiobase 3.4/impacts_FR.pkl', 'wb'))

### Aggregation of sectors into broad categories

In [4]:
def consumption_sectors(self, notion): 
    '''
    Returns the list of sectors (or names) corresponding to notion, which can be: food, drug, cloth, housing, furniture, health, transport, communication, leisure, education, catering, diverse
    '''
    if notion is None or notion == 'total': sectors = self.sectors
    elif notion not in ['food', 'tobacco', 'cloth', 'housing', 'furniture', 'health', 'transport', 'communication', 'leisure', 'education', 'catering', 'diverse']:
        print('Error: notion unknown')
    elif notion=='food': sectors = ['Paddy rice',  'Wheat',  'Cereal grains nec',  'Vegetables, fruit, nuts',  'Oil seeds',  'Sugar cane, sugar beet',  'Plant-based fibers',  'Crops nec',  'Cattle',  'Pigs',  'Poultry',  'Meat animals nec',  'Animal products nec',  'Raw milk', 'Fish and other fishing products; services incidental of fishing (05)', 
'Products of meat cattle', 'Products of meat pigs', 'Products of meat poultry', 'Meat products nec', 'products of Vegetable oils and fats', 'Dairy products', 'Processed rice', 'Sugar', 'Food products nec', 'Beverages', 'Fish products']
    elif notion=='tobacco': sectors = ['Tobacco products (16)']
    elif notion=='cloth': sectors = ['Textiles (17)', 'Wearing apparel; furs (18)', 'Leather and leather products (19)']
    elif notion=='housing': sectors = ['Electricity by coal', 'Electricity by gas', 'Electricity by nuclear', 'Electricity by hydro', 'Electricity by wind', 'Electricity by petroleum and other oil derivatives', 'Electricity by biomass and waste', 'Electricity by solar photovoltaic', 'Electricity by solar thermal', 'Electricity by tide, wave, ocean', 'Electricity by Geothermal', 'Electricity nec', 'Steam and hot water supply services', 'Collected and purified water, distribution services of water (41)', 'Construction work (45)', 'Real estate services (70)']
    elif notion=='furniture': sectors = ['Paper and paper products', 'Printed matter and recorded media (22)', 'Furniture; other manufactured goods n.e.c. (36)'] 
    elif notion=='health': sectors = ['Health and social work services (85)'] 
    elif notion=='transport': sectors = ['Motor Gasoline', 'Gas/Diesel Oil', 'Heavy Fuel Oil', 'Refinery Gas', 'Liquefied Petroleum Gases (LPG)', 'Kerosene', 'Motor vehicles, trailers and semi-trailers (34)', 'Other transport equipment (35)', 'Sale, maintenance, repair of motor vehicles, motor vehicles parts, motorcycles, motor cycles parts and accessoiries', 'Retail trade services of motor fuel', 'Railway transportation services', 'Other land transportation services', 'Transportation services via pipelines', 'Sea and coastal water transportation services', 'Inland water transportation services', 'Air transport services (62)', 'Supporting and auxiliary transport services; travel agency services (63)'] 
    elif notion=='communication': sectors = ['Office machinery and computers (30)', 'Electrical machinery and apparatus n.e.c. (31)', 'Radio, television and communication equipment and apparatus (32)', 'Post and telecommunication services (64)'] 
    elif notion=='leisure': sectors = ['Recreational, cultural and sporting services (92)'] 
    elif notion=='education': sectors = ['Education services (80)'] 
    elif notion=='catering': sectors = ['Hotel and restaurant services (55)'] 
    elif notion=='diverse': sectors = ['Fabricated metal products, except machinery and equipment (28)', 'Machinery and equipment n.e.c. (29)', 'Wholesale trade and commission trade services, except of motor vehicles and motorcycles (51)', 'Retail  trade services, except of motor vehicles and motorcycles; repair services of personal and household goods (52)', 
'Wool, silk-worm cocoons',  'Manure (conventional treatment)',  'Manure (biogas treatment)',  'Products of forestry, logging and related services (02)', 'Anthracite',  'Coking Coal',  'Other Bituminous Coal',  'Sub-Bituminous Coal',  'Patent Fuel',  'Lignite/Brown Coal',  'BKB/Peat Briquettes',  'Peat', 'Natural Gas Liquids', 'Other Hydrocarbons', 'Uranium and thorium ores (12)', 'Iron ores', 'Copper ores and concentrates', 'Nickel ores and concentrates', 'Aluminium ores and concentrates', 'Precious metal ores and concentrates', 'Lead, zinc and tin ores and concentrates', 'Other non-ferrous metal ores and concentrates', 'Stone', 'Sand and clay', 'Chemical and fertilizer minerals, salt and other mining and quarrying products n.e.c.', 'Wood and products of wood and cork (except furniture); articles of straw and plaiting materials (20)', 'Wood material for treatment, Re-processing of secondary wood material into new wood material', 'Pulp', 'Secondary paper for treatment, Re-processing of secondary paper into new pulp', 'Crude petroleum and services related to crude oil extraction, excluding surveying', 'Natural gas and services related to natural gas extraction, excluding surveying', 'Coke Oven Coke', 'Gas Coke', 'Coal Tar', 'Aviation Gasoline', 'Gasoline Type Jet Fuel', 'Kerosene Type Jet Fuel', 'Refinery Feedstocks', 'Ethane', 'Naphtha', 'White Spirit & SBP', 'Lubricants', 'Bitumen', 'Paraffin Waxes', 'Petroleum Coke', 'Non-specified Petroleum Products', 'Nuclear fuel', 'Plastics, basic', 'Secondary plastic for treatment, Re-processing of secondary plastic into new plastic', 'N-fertiliser', 'P- and other fertiliser', 'Chemicals nec', 'Charcoal', 'Additives/Blending Components', 'Biogasoline', 'Biodiesels', 'Other Liquid Biofuels', 'Rubber and plastic products (25)', 'Glass and glass products', 'Secondary glass for treatment, Re-processing of secondary glass into new glass', 'Ceramic goods', 'Bricks, tiles and construction products, in baked clay', 'Cement, lime and plaster', 'Ash for treatment, Re-processing of ash into clinker', 'Other non-metallic mineral products', 'Basic iron and steel and of ferro-alloys and first products thereof', 'Secondary steel for treatment, Re-processing of secondary steel into new steel', 'Precious metals', 'Secondary preciuos metals for treatment, Re-processing of secondary preciuos metals into new preciuos metals', 'Aluminium and aluminium products', 'Secondary aluminium for treatment, Re-processing of secondary aluminium into new aluminium', 'Lead, zinc and tin and products thereof', 'Secondary lead for treatment, Re-processing of secondary lead into new lead', 'Copper products', 'Secondary copper for treatment, Re-processing of secondary copper into new copper', 'Other non-ferrous metal products', 'Secondary other non-ferrous metals for treatment, Re-processing of secondary other non-ferrous metals into new other non-ferrous metals', 'Foundry work services', 'Medical, precision and optical instruments, watches and clocks (33)', 'Secondary raw materials', 'Bottles for treatment, Recycling of bottles by direct reuse', 'Transmission services of electricity', 'Distribution and trade services of electricity', 'Coke oven gas', 'Blast Furnace Gas', 'Oxygen Steel Furnace Gas', 'Gas Works Gas', 'Biogas', 'Distribution services of gaseous fuels through mains', 'Secondary construction material for treatment, Re-processing of secondary construction material into aggregates', 'Financial intermediation services, except insurance and pension funding services (65)', 'Insurance and pension funding services, except compulsory social security services (66)', 'Services auxiliary to financial intermediation (67)', 'Renting services of machinery and equipment without operator and of personal and household goods (71)', 'Computer and related services (72)', 'Research and development services (73)', 'Other business services (74)', 'Public administration and defence services; compulsory social security services (75)', 'Food waste for treatment: incineration', 'Paper waste for treatment: incineration', 'Plastic waste for treatment: incineration', 'Intert/metal waste for treatment: incineration', 'Textiles waste for treatment: incineration', 'Wood waste for treatment: incineration', 'Oil/hazardous waste for treatment: incineration', 'Food waste for treatment: biogasification and land application', 'Paper waste for treatment: biogasification and land application', 'Sewage sludge for treatment: biogasification and land application', 'Food waste for treatment: composting and land application', 'Paper and wood waste for treatment: composting and land application', 'Food waste for treatment: waste water treatment', 'Other waste for treatment: waste water treatment', 'Food waste for treatment: landfill', 'Paper for treatment: landfill', 'Plastic waste for treatment: landfill', 'Inert/metal/hazardous waste for treatment: landfill', 'Textiles waste for treatment: landfill', 'Wood waste for treatment: landfill', 'Membership organisation services n.e.c. (91)', 'Other services (93)', 'Private households with employed persons (95)', 'Extra-territorial organizations and bodies'] 
    elif notion=='undecided': sectors = ['Beverages', 'Paper and paper products', 'Printed matter and recorded media (22)']
    return(sectors)

IOS.consumption_sectors = consumption_sectors

In [5]:
sectors = ['food', 'tobacco', 'cloth', 'housing', 'furniture', 'health', 'transport', 'communication', 'leisure', 'education', 'catering', 'diverse']
imp = exio3.impact.F.index

### Aggregation of impacts under broad categories (e.g. all CO2 emissions)

In [22]:
pd.set_option('display.max_rows', 2000)
exio3.impact.unit

In [10]:
impacts = ['CO2 (kg)', 'CH4 (kg)', 'NOx (kg)', 'SOx (kg)', 'N2O (kg)', 'NH3 (kg)', 'CO (kg)', 'PFC (kg CO2eq)', 'HFC (kg CO2eq)', 'SF6 (kg)',
         'PM10 (kg)', 'PM2.5 (kg)', 'Pb (kg)', 'Cd (kg)', 'Gravel and sand (kg)',
         'Arsenic (kg)', 'NMVOC (kg)', 'TSP (kg)', 'N (kg)', 'P (kg)', 'Energy (TJ)', 'Wood (kt)', 'Copper (kt)', 'Metal Ores (kt)', 'Non-Metallic Minerals (kt)', 
         'Crops (kt)', 'Cropland (km2)', 'Forest area (km2)', 'Permanent pastures (km2)', 'Land use (km2)', 'Water Consumption Blue (Mm3)', 
         'Water Consumption Green (Mm3)', 'Water Withdrawal Blue (Mm3)', 'Wages (incl. social contributions) (M€)', 
         'Operating surplus (investment, rents, profit) (M€)', 'Value added (wages, surplus and taxes) (M€)', 'Employment (k pers)', 'Employment hours (M.hr)']

In [6]:
agg_impacts = exio3.impact.F
exio3.impact.F = agg_impacts.append([ # TODO: énergie, déchets, minerais, bois TODO: Global Warming Potential (CO2eq)
#     pd.Series(agg_impacts.iloc[['CO2 - combustion - air' in i for i in imp],:].sum(axis = 0), name = 'CO2 - combustion - air (kg)'),
    pd.Series(agg_impacts.iloc[['CO2' in i for i in imp],:].sum(axis = 0), name = 'CO2 (kg)'),
    pd.Series(agg_impacts.iloc[['CH4' in i for i in imp],:].sum(axis = 0), name = 'CH4 (kg)'),
    pd.Series(agg_impacts.iloc[['NOx' in i for i in imp],:].sum(axis = 0), name = 'NOx (kg)'),
    pd.Series(agg_impacts.iloc[['SOx' in i for i in imp],:].sum(axis = 0), name = 'SOx (kg)'),
    pd.Series(agg_impacts.iloc[['N2O' in i for i in imp],:].sum(axis = 0), name = 'N2O (kg)'),
    pd.Series(agg_impacts.iloc[['NH3' in i for i in imp],:].sum(axis = 0), name = 'NH3 (kg)'),
    pd.Series(agg_impacts.iloc[['CO -' in i for i in imp],:].sum(axis = 0), name = 'CO (kg)'),
    pd.Series(agg_impacts.iloc[['PFC' in i for i in imp],:].sum(axis = 0), name = 'PFC (kg CO2eq)'),
    pd.Series(agg_impacts.iloc[['HFC' in i for i in imp],:].sum(axis = 0), name = 'HFC (kg CO2eq)'),
    pd.Series(agg_impacts.iloc[['SF6' in i for i in imp],:].sum(axis = 0), name = 'SF6 (kg)'),
    pd.Series(agg_impacts.iloc[['PM10' in i for i in imp],:].sum(axis = 0), name = 'PM10 (kg)'),
    pd.Series(agg_impacts.iloc[['PM2.5' in i for i in imp],:].sum(axis = 0), name = 'PM2.5 (kg)'),
    pd.Series(agg_impacts.iloc[['Pb' in i for i in imp],:].sum(axis = 0), name = 'Pb (kg)'),
    pd.Series(agg_impacts.iloc[['Cd' in i for i in imp],:].sum(axis = 0), name = 'Cd (kg)'),
    pd.Series(agg_impacts.iloc[['sand' in i for i in imp],:].sum(axis = 0), name = 'Gravel and sand (kg)'),
    pd.Series(agg_impacts.iloc[['As -' in i for i in imp],:].sum(axis = 0), name = 'Arsenic (kg)'),
    pd.Series(agg_impacts.iloc[['NMVOC' in i for i in imp],:].sum(axis = 0), name = 'NMVOC (kg)'),
    pd.Series(agg_impacts.iloc[['TSP' in i for i in imp],:].sum(axis = 0), name = 'TSP (kg)'),
    pd.Series(agg_impacts.iloc[['N -' in i for i in imp],:].sum(axis = 0), name = 'N (kg)'),
    pd.Series(agg_impacts.iloc[['P - a' in i or 'P - w' in i for i in imp],:].sum(axis = 0), name = 'P (kg)'),
    pd.Series(agg_impacts.iloc[['Energy Carrier Supply' in i for i in imp],:].sum(axis = 0), name = 'Energy (TJ)'),
    pd.Series(agg_impacts.iloc[['s wood -' in i for i in imp],:].sum(axis = 0), name = 'Wood (kt)'), # from Wood to Crops, includes both used and unused extraction
    pd.Series(agg_impacts.iloc[['Ores - Copper' in i for i in imp],:].sum(axis = 0), name = 'Copper (kt)'),
    pd.Series(agg_impacts.iloc[['Metal Ores' in i for i in imp],:].sum(axis = 0), name = 'Metal Ores (kt)'),
    pd.Series(agg_impacts.iloc[['Non-Metallic Minerals' in i for i in imp],:].sum(axis = 0), name = 'Non-Metallic Minerals (kt)'),
    pd.Series(agg_impacts.iloc[['Primary Crops' in i for i in imp],:].sum(axis = 0), name = 'Crops (kt)'),
    pd.Series(agg_impacts.iloc[['Cropland' in i for i in imp],:].sum(axis = 0), name = 'Cropland (km2)'),
    pd.Series(agg_impacts.iloc[['Forest area' in i for i in imp],:].sum(axis = 0), name = 'Forest area (km2)'),
    pd.Series(agg_impacts.iloc[['Permanent pasture' in i for i in imp],:].sum(axis = 0), name = 'Permanent pastures (km2)'),
    pd.Series(agg_impacts.iloc[['Cropland' in i or 'land Use' in i or 'pasture' in i for i in imp],:].sum(axis = 0), name = 'Land use (km2)'),
    pd.Series(agg_impacts.iloc[['Water Consumption Blue' in i for i in imp],:].sum(axis = 0), name = 'Water Consumption Blue (Mm3)'),
    pd.Series(agg_impacts.iloc[['Water Consumption Green' in i for i in imp],:].sum(axis = 0), name = 'Water Consumption Green (Mm3)'),
    pd.Series(agg_impacts.iloc[['Water Withdrawal' in i for i in imp],:].sum(axis = 0), name = 'Water Withdrawal Blue (Mm3)'),
    pd.Series(agg_impacts.iloc[['wages' in i for i in imp],:].sum(0), name='Wages (incl. social contributions) (M€)'),
    pd.Series(agg_impacts.iloc[['surplus' in i for i in imp],:].sum(0), name='Operating surplus (investment, rents, profit) (M€)'),
    pd.Series(agg_impacts.iloc[['axes' in i or 'wages' in i or 'surplus' in i for i in imp],:].sum(0), name='Value added (wages, surplus and taxes) (M€)'),
    pd.Series(agg_impacts.iloc[['Employment:' in i and 'Vulnerable' not in i for i in imp],:].sum(0), name='Employment (k pers)'),
    pd.Series(agg_impacts.iloc[['Employment hours:' in i and 'Vulnerable' not in i for i in imp],:].sum(0), name = 'Employment hours (M.hr)')])
del(agg_impacts)

### Computation of French footprints by sector

In [ ]:
impacts_FR = pd.DataFrame(index = sectors, columns = impacts + ['GWP 100 (kg CO2eq)'])
for impact in impacts: # , 'GWP 100 (kg CO2eq)', 'GWP 500 (kg CO2eq)', 'GWP 20 with feedback (kg CO2eq)', 'GWP 100 with feedback (kg CO2eq)'
    for secs in sectors:
        impacts_FR.at[secs,impact] = exio3.embodied_impact(regs = 'FR', secs = exio3.consumption_sectors(secs), var = impact).sum()

In [ ]:
for secs in sectors: # Source: https://en.wikipedia.org/wiki/Global_warming_potential (21/03/2020). Note that CF4=PFC.
    # I assume that the period is 100 years (for consistence with PFC and HFC), that HFC=HFC-134a, and I don't account for PFTBA nor CFC (absent from data).
    impacts_FR.at[secs, 'GWP 100 (kg CO2eq)'] =    (impacts_FR.at[secs, 'CO2 (kg)'] + 
                                                 28*impacts_FR.at[secs, 'CH4 (kg)'] + 
                                                268*impacts_FR.at[secs, 'N2O (kg)'] +
                                                    impacts_FR.at[secs, 'HFC (kg CO2eq)'] + 
                                                    impacts_FR.at[secs, 'PFC (kg CO2eq)'] + 
                                              22200*impacts_FR.at[secs, 'SF6 (kg)'])
#     impacts_FR.at[secs, 'GWP 20 (kg CO2eq)'] = (impacts_FR.at[secs, 'CO2 (kg)'] + 84*impacts_FR.at[secs, 'CH4 (kg)'] + 264*impacts_FR.at[secs, 'N2O (kg)']
#         + impacts_FR.at[secs, 'HFC (kg CO2eq)'] + 6900*impacts_FR.at[secs, 'CFC (kg)'] +  impacts_FR.at[secs, 'PFC (kg CO2eq)'] + 15100*impacts_FR.at[secs, 'SF6 (kg)'])
#     impacts_FR.at[secs, 'GWP 500 (kg CO2eq)'] = (impacts_FR.at[secs, 'CO2 (kg)'] + 7.6*impacts_FR.at[secs, 'CH4 (kg)'] + 153*impacts_FR.at[secs, 'N2O (kg)']
#         + impacts_FR.at[secs, 'HFC (kg CO2eq)'] + 1620*impacts_FR.at[secs, 'CFC (kg)'] +  impacts_FR.at[secs, 'PFC (kg CO2eq)'] + 32600*impacts_FR.at[secs, 'SF6 (kg)'])
#     impacts_FR.at[secs, 'GWP 20 with feedback (kg CO2eq)'] =(impacts_FR.at[secs, 'CO2 (kg)']+86*impacts_FR.at[secs, 'CH4 (kg)']+268*impacts_FR.at[secs, 'N2O (kg)']
#         + impacts_FR.at[secs, 'HFC (kg CO2eq)'] + 7020*impacts_FR.at[secs, 'CFC (kg)'] + impacts_FR.at[secs, 'PFC (kg CO2eq)'] + 16300*impacts_FR.at[secs, 'SF6 (kg)'])
#     impacts_FR.at[secs, 'GWP 100 with feedback (kg CO2eq)']=(impacts_FR.at[secs, 'CO2 (kg)']+34*impacts_FR.at[secs, 'CH4 (kg)']+298*impacts_FR.at[secs, 'N2O (kg)']
#         + impacts_FR.at[secs, 'HFC (kg CO2eq)'] + 5350*impacts_FR.at[secs, 'CFC (kg)'] + impacts_FR.at[secs, 'PFC (kg CO2eq)'] + 22800*impacts_FR.at[secs, 'SF6 (kg)'])

In [ ]:
impacts_FR = impacts_FR.append(impacts_FR.sum().rename('Total'))

In [ ]:
impacts_FR

### Computation of impact by sector and age

In [ ]:
conso_age = pd.DataFrame(index = sectors, columns = range(100))
for sec in sectors: conso_age.at[sec,:] = pd.read_excel(path_dl + 'conso_age_FR_2011.xlsx', header = None, sheet_name = sec).iloc[:,0]
conso_age = conso_age.div(conso_age.sum(1), axis=0) # at i,j: share of conso of sector i by age j (so that sum of each row = 1)

In [ ]:
impacts_age = pd.DataFrame(index = sectors, columns = pd.MultiIndex.from_product([impacts, list(range(100))], names=('impact', 'age')))
for sec in sectors:
    for i in impacts:
        impacts_age.loc[sec, [(i, a) for a in range(100)]] = np.array(conso_age.loc[sec] * impacts_FR.at[sec, i])
impacts_age = impacts_age.append(impacts_age.sum().rename('Total'))

In [ ]:
impacts_age

### Export

In [41]:
with pd.ExcelWriter(path_dl + 'impacts_age_FR_2011.xlsx') as writer:  
    for i in impacts: 
        if len(i) < 32: name = i
        elif i == 'Wages (incl. social contributions) (M€)': name = 'Wages (M€)'
        elif i == 'Operating surplus (investment, rents, profit) (M€)': name = 'Operating surplus (M€)'
        elif i == 'Value added (wages, surplus and taxes) (M€)': name = 'Value added (M€)'
        else: print(i)
        impacts_age[i].to_excel(writer, sheet_name = name) 

In [42]:
pickle.dump(impacts_age, open(path_io + 'Exiobase 3.4/agg_impacts_age.pkl', 'wb'))

In [33]:
pickle.dump(impacts_FR, open(path_io + 'Exiobase 3.4/impacts_FR.pkl', 'wb'))

In [13]:
pickle.dump(exio3, open(path_io + 'Exiobase 3.4/exio3.pkl', 'wb'))

### Aggregation ex ante: biases the results (e.g. underestimate CO2 emissions by 20%)

In [4]:
# exio3 = parse_exiobase3(path_io+'Exiobase 3.4/exiobase3.4_iot_2011_pxp.zip')
# exio3.calc_all()
# exio3.x = exio3.x.squeeze()
# exio3.impact = exio3.satellite

In [42]:
# agg_exio3 = pickle.load(open(path_io+'Exiobase 3.4/agg_exio3.pkl', 'rb'))

In [35]:
# agg_region = [1 if x=='FR' else 0 for x in exio3.regions]
# agg_sector = []
# for s in exio3.sectors:
#     for i, sec in enumerate(sectors):  
#         if s in exio3.consumption_sectors(sec):
#             agg_sector = agg_sector + [i]

In [8]:
# agg_exio3 = exio3.aggregate(region_agg = agg_region, sector_agg = agg_sector, region_names = ['RoW', 'FR'], sector_names = sectors, inplace = False) 
# agg_exio3.calc_all()
# agg_exio3.x = agg_exio3.x.iloc[:,0]

In [9]:
# agg_impacts = agg_exio3.impact.F
# agg_exio3.impact.F = agg_impacts.append([ 
# #     pd.Series(agg_impacts.iloc[['CO2 - combustion - air' in i for i in imp],:].sum(axis = 0), name = 'CO2 - combustion - air (kg)'),
#     pd.Series(agg_impacts.iloc[['CO2' in i for i in imp],:].sum(axis = 0), name = 'CO2 (kg)'),
#     pd.Series(agg_impacts.iloc[['CH4' in i for i in imp],:].sum(axis = 0), name = 'CH4 (kg)'),
#     pd.Series(agg_impacts.iloc[['NOx' in i for i in imp],:].sum(axis = 0), name = 'NOx (kg)'),
#     pd.Series(agg_impacts.iloc[['SOx' in i for i in imp],:].sum(axis = 0), name = 'SOx (kg)'),
#     pd.Series(agg_impacts.iloc[['N2O' in i for i in imp],:].sum(axis = 0), name = 'N2O (kg)'),
#     pd.Series(agg_impacts.iloc[['NH3' in i for i in imp],:].sum(axis = 0), name = 'NH3 (kg)'),
#     pd.Series(agg_impacts.iloc[['CO -' in i for i in imp],:].sum(axis = 0), name = 'CO (kg)'),
#     pd.Series(agg_impacts.iloc[['PM10' in i for i in imp],:].sum(axis = 0), name = 'PM10 (kg)'),
#     pd.Series(agg_impacts.iloc[['PM2.5' in i for i in imp],:].sum(axis = 0), name = 'PM2.5 (kg)'),
#     pd.Series(agg_impacts.iloc[['Pb' in i for i in imp],:].sum(axis = 0), name = 'Pb (kg)'),
#     pd.Series(agg_impacts.iloc[['Cd' in i for i in imp],:].sum(axis = 0), name = 'Cd (kg)'),
#     pd.Series(agg_impacts.iloc[['Cropland' in i for i in imp],:].sum(axis = 0), name = 'Cropland (km2)'),
#     pd.Series(agg_impacts.iloc[['Forest area' in i for i in imp],:].sum(axis = 0), name = 'Forest area (km2)'),
#     pd.Series(agg_impacts.iloc[['Permanent pasture' in i for i in imp],:].sum(axis = 0), name = 'Permanent pastures (km2)'),
#     pd.Series(agg_impacts.iloc[['Cropland' in i or 'land Use' in i or 'pasture' in i for i in imp],:].sum(axis = 0), name = 'Land use (km2)'),
#     pd.Series(agg_impacts.iloc[['Water Consumption Blue' in i for i in imp],:].sum(axis = 0), name = 'Water Consumption Blue (Mm3)'),
#     pd.Series(agg_impacts.iloc[['Water Consumption Green' in i for i in imp],:].sum(axis = 0), name = 'Water Consumption Green (Mm3)'),
#     pd.Series(agg_impacts.iloc[['Water Withdrawal' in i for i in imp],:].sum(axis = 0), name = 'Water Withdrawal Blue (Mm3)'),
#     pd.Series(agg_impacts.iloc[['wages' in i for i in imp],:].sum(0), name='Wages (incl. social contributions) (M€)'),
#     pd.Series(agg_impacts.iloc[['surplus' in i for i in imp],:].sum(0), name='Operating surplus (investment, rents, profit) (M€)'),
#     pd.Series(agg_impacts.iloc[['axes' in i or 'wages' in i or 'surplus' in i for i in imp],:].sum(0), name='Value added (wages, surplus and taxes) (M€)'),
#     pd.Series(agg_impacts.iloc[['Employment:' in i and 'Vulnerable' not in i for i in imp],:].sum(0), name='Employment (k pers)'),
#     pd.Series(agg_impacts.iloc[['Employment hours:' in i and 'Vulnerable' not in i for i in imp],:].sum(0), name = 'Employment hours (M.hr)')])
# del(agg_impacts)
# agg_exio3.impact.F = agg_exio3.impact.F.drop(agg_exio3.impact.F.index[range(len(imp))])

In [31]:
# pd.set_option('display.max_rows', 25)
# agg_exio3.impact.F

In [15]:
# agg_impacts_FR = pd.DataFrame(index = sectors, columns = impacts)
# for impact in impacts:
#     for secs in sectors:
#         agg_impacts_FR.at[secs, impact] = agg_exio3.embodied_impact(regs = 'FR', secs = secs, var = impact).sum()
# agg_impacts_FR = agg_impacts_FR.append(agg_impacts_FR.sum().rename('Total'))

In [27]:
# agg_impacts_FR

In [ ]:
# pickle.dump(agg_exio3, open(path_io + 'Exiobase 3.4/agg_exio3.pkl', 'wb'))

In [17]:
# pickle.dump(agg_impacts_FR, open(path_io + 'Exiobase 3.4/impacts_FR.pkl', 'wb'))